Импорт библиотек

In [1002]:
import pandas as pd
import numpy as np
import pickle

Загрузка файлов

In [1003]:
# Загрузка моделей
with open('models/catboost_model.pkl', 'rb') as f:
    catboost_model = pickle.load(f)

with open('models/xgboost_model.pkl', 'rb') as f:
    xgboost_model = pickle.load(f)

with open('models/randomforest_model.pkl', 'rb') as f:
    randomforest_model = pickle.load(f)

# Загрузка тестовых данных
test_data = pd.read_excel('data/test_prepared.xlsx')

Прогнозирование

In [1004]:
# Подготовка данных для теста
X_test = test_data.drop(columns=["dt", "Цена на арматуру"])
y_test = test_data["Цена на арматуру"]

# Получение предсказаний для всех моделей
y_pred_catboost = catboost_model.predict(X_test)
y_pred_xgboost = xgboost_model.predict(X_test)
y_pred_rf = randomforest_model.predict(X_test)

# Округление предсказанных значений и сохранение в массив
y_pred_catboost = np.round(y_pred_catboost).astype(int)
y_pred_xgboost = np.round(y_pred_xgboost).astype(int)
y_pred_rf = np.round(y_pred_rf).astype(int)

# Массив предсказанных значений
predictions = [y_pred_catboost, y_pred_xgboost, y_pred_rf]
model_names = ['CatBoost', 'XGBoost', 'RandomForest']

# Функция для рекомендации объема тендера
def main_recommend_tender_volume_(predictions, model_names, test_data):
    def perenos_():
        print('\n******************************************************************\n')

    def recommend_tender_volume(predicted_price, current_price):
        if predicted_price < current_price * 0.95:
            return max(1, int((current_price - predicted_price) / 1000))
        elif predicted_price > current_price * 1.05:
            return max(6, int((predicted_price - current_price) / 1000))
        else:
            return 3

    current_price = test_data['Цена на арматуру'].iloc[-1]  # цена на арматуру сегодня

    for preds, model_name in zip(predictions, model_names):
        recommended_volumes = []

        # Вычисление тендера
        for predicted_price in preds:
            recommended_volume = recommend_tender_volume(predicted_price, current_price)
            recommended_volumes.append(recommended_volume)

        differences = preds - y_test.values
        
        # Проверка предсказанных значений
        print(f"Рекомендуемый объем тендера на арматуру для тестовых данных на основе {model_name}: {recommended_volumes}")
        perenos_()
        print(f"Предсказанные цены на тестовом наборе для {model_name}: {preds}\n")
        print(f"Фактические цены на тестовом наборе: {y_test.values}\n")
        print(f"Разницы между предсказанными и фактическими значениями: {differences}\n")

        # Добавление рекомендаций в test_data
        test_data[f'Объём {model_name}'] = recommended_volumes
        print(f"Столбец объёма {model_name}, успешно добавлен!")
        perenos_()

# Вызов функции
main_recommend_tender_volume_(predictions, model_names, test_data)

Рекомендуемый объем тендера на арматуру для тестовых данных на основе CatBoost: [9, 10, 11, 12, 12, 16, 16, 15, 14, 12, 12, 12, 14, 13, 13, 13, 13, 12, 11, 9, 6, 3, 3]

******************************************************************

Предсказанные цены на тестовом наборе для CatBoost: [45644 44999 44401 43342 43474 39314 39183 40073 40815 42759 42634 42634
 41004 41781 41700 41781 41781 43132 44011 46222 49445 53719 54603]

Фактические цены на тестовом наборе: [45000 44000 43000 42000 41500 41000 40000 40000 40500 41700 41700 42500
 42700 42700 42700 42800 42800 44500 44500 46000 49200 53700 55500]

Разницы между предсказанными и фактическими значениями: [  644   999  1401  1342  1974 -1686  -817    73   315  1059   934   134
 -1696  -919 -1000 -1019 -1019 -1368  -489   222   245    19  -897]

Столбец объёма CatBoost, успешно добавлен!

******************************************************************

Рекомендуемый объем тендера на арматуру для тестовых данных на основе XGBoost: [

Сохранение результатов

In [1005]:
# Оставляем только нужные столбцы
columns_to_keep = ['dt', 'Цена на арматуру', 'Объём CatBoost', 'Объём XGBoost', 'Объём RandomForest']
test_data_filtered = test_data[columns_to_keep]

# Сохранение обновленного DataFrame в test.xlsx
test_data_filtered.to_excel('data/test.xlsx', index=False)

1. CatBoost: Модель показывает хорошую производительность, но имеются значительные расхождения в предсказаниях, особенно в случае высоких разниц.

2. XGBoost: Модель работает достаточно хорошо, с меньшими расхождениями, чем CatBoost, что указывает на её эффективность.

3. RandomForest: Модель также показывает хорошие результаты, но наблюдаются большие расхождения, особенно в некоторых предсказаниях.

Вывод: XGBoost лучшая модель